In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
from forward import *

In [3]:
alpha, theta, radius, dists, active, limits = get_DH_params(preset_models["HangingArm"])

In [4]:
x = interface(alpha, theta, radius, dists, active, limits)

In [5]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [164]:
torch.save(model, "HangingArm")

In [7]:
model = torch.load("HangingArm")

In [172]:
animate_robot(alpha, theta, radius, dists, active, limits)

ValueError: too many values to unpack (expected 3)

In [9]:
For_model.orientation = True
inputs, target = For_model.generate_maps()
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=5, depth=20)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
inputs, target = For_model.generate_maps()

In [30]:
model2 = model

In [33]:
train_positions(For_model, model2)

253.51
260.37
249.78


KeyboardInterrupt: 

In [13]:
For_model.orientation = False

In [35]:
evaluate_plot(For_model, model2)

tensor(282.2444, grad_fn=<PowBackward0>)


In [36]:
train_motors(For_model, model)

-------------
200.58640518188477
43.686625480651855
18.401508712768553
15.15464506149292
12.624349880218507
-------------
11.244905948638916
12.811871337890626
11.488945960998535
11.212309837341309
10.863507747650146
-------------
11.937852096557616
12.12466516494751
10.414449214935303
11.451369190216065
11.486302089691161
-------------
12.696941566467284
11.192619514465331
11.632076072692872
12.046437454223632
11.32234058380127
-------------
13.758905410766602
13.102630615234375
11.594927406311035
12.8140700340271
12.572765159606934
-------------
12.84487009048462
11.23971185684204
12.073593807220458
12.395498847961425
10.817351245880127
-------------
12.3349440574646
11.433049774169922
11.326505947113038
12.015381622314454
10.959171485900878
-------------
12.371792125701905
11.37380895614624
11.056385231018066
10.602121067047118
12.794064140319824
-------------
13.032036304473877
12.253183937072754
11.605185604095459
12.791501235961913
11.466855716705322
-------------
11.457996559143

10.49926118850708
9.960103082656861
9.848999214172363
10.249166297912598
11.17192211151123
-------------
9.64235486984253
9.544299125671387
10.920340156555175
9.095831871032715
8.424434041976928
-------------
11.160824584960938
10.53608808517456
10.378893375396729
11.970452690124512
10.820545768737793
-------------
12.866477489471436
11.564229774475098
11.836244773864745
11.375897884368896
9.802266788482665
-------------
10.720308589935303
11.404511260986329
10.651309871673584
11.063099193572999
10.538792133331299
-------------
9.727980756759644
10.308766174316407
10.354406356811523
9.922149753570556
10.74967041015625
-------------
13.492716884613037
13.301612377166748
12.920339679718017
11.929085063934327
11.627874946594238
-------------
11.990170669555663
11.80424337387085
11.598129367828369
10.807856559753418
11.629508972167969
-------------
11.218714714050293
11.503940391540528
10.234634113311767
9.840929222106933
11.33818416595459
-------------
11.223449230194092
10.69960899353027

In [25]:
inputs, target = For_model.generate_maps(n=1)
#mot_pos = np.array([alpha,theta,radius,dists])
n= np.random.randint(len(inputs))
#######################
pred = model(torch.Tensor(inputs[n:n+1])).detach()
######################
For_model.set_active(pred[0])
For_model.update_position()
pos_pred = For_model.run_forward().round(2)
#######################
For_model.set_active(target[n])
pos_targ = For_model.run_forward().round(2)
########################
print("--- x y z ---")
print(inputs[n].round(2))
print(pos_targ)
print(pos_pred )
print("--- torch ---")
pred_xys = with_torch().forward_from_active(For_model, target[n:n+1])
print(pred_xys[0,0].numpy().round(2))
pred_xys = with_torch().forward_from_active(For_model, pred)
print(pred_xys[0,0].numpy().round(2))
print("--- motor positions ---")
print(target[n].round(2))
print(pred.numpy().round(2)[0])

--- x y z ---
[ 32.   126.06 101.53 105.85 214.62 279.16]
[ 32.   126.06 101.53 105.85 214.62 279.16]
[ 29.97 127.84  98.73 106.71 214.42 279.63]
--- torch ---
[ 32.   126.06 101.53 105.85 214.62 279.16]
[ 29.97 127.84  98.73 106.71 214.42 279.63]
--- motor positions ---
[170.65 144.33  41.56  61.43]
[169.73 144.42  44.64  59.08]


In [267]:
def get_DH(For_model):
    alpha  = torch.tensor(1.*np.array(For_model.alpha)) 
    theta  = torch.tensor(1.*np.array(For_model.theta)) 
    radius = torch.tensor(1.*np.array(For_model.radius))
    dists  = torch.tensor(1.*np.array(For_model.dists)) 
    active = For_model.active
    DH = torch.stack([alpha, theta, radius, dists])
    return DH

def get_transform(DH):
    alpha, theta, radius, dists = DH
    alpha = torch.deg2rad(alpha)
    theta = torch.deg2rad(theta)
    
    TFs = make_transforms(alpha, theta, radius, dists)
    return TFs[-1]

def make_transforms(alpha, theta, radius, dists):

    mot_params = torch.stack([alpha, theta, radius, dists])

    Tfs = []
    T = torch.eye(4)
    for i in range(mot_params.shape[1]):
        a,t,r,d = mot_params[:,i]
        DHt = DH_transform(a,t,r,d) 
        T = torch.matmul(T,DHt)     
        Tfs.append(T)

    return Tfs

def DH_transform(a,t,r,z):
        Zt = Z_transform(a,t,r,z)
        Xt = X_transform(a,t,r,z)
        DHt = torch.matmul(Zt,Xt)
        return DHt

    ###############################################
def X_transform(a,t,r,d):

    Xt = torch.eye(4)
    Xt[0,3] = r
    Xt[1,1] = torch.cos(a)
    Xt[1,2] = -1*torch.sin(a)
    Xt[2,1] = torch.sin(a)
    Xt[2,2] = torch.cos(a)

    return Xt

def Z_transform(a,t,r,d):

    Zt = torch.eye(4)
    Zt[0,0] = torch.cos(t)
    Zt[0,1] = -1*torch.sin(t)
    Zt[1,0] = torch.sin(t)
    Zt[1,1] = torch.cos(t)
    Zt[2,3] = d

    return Zt

In [313]:
matrix = torch.Tensor(np.eye(3))

In [327]:
rot_angle = torch_matrix_to_rotation( matrix )
rot_angle

tensor([1.5708, 1.0472, 3.1416])

In [303]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

In [304]:
rotation = np.array(rotation)

tensor(-0.1879)

In [326]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

matrix = torch.Tensor(R.from_euler("XYZ",[np.pi/2,np.pi/3,np.pi]).as_matrix())

In [346]:
for _ in range(100):
    rot = np.random.uniform(np.pi,size=(3))
    out = R.from_matrix(R.from_euler("XYZ",rot).as_matrix()).as_euler("XYZ")
    if out[0]<0:  
        out[[0,2]] = out[[0,2]] + np.pi
        out[[1]]   = np.pi - out[[1]] 
    print(rot, out, rot.round(4) == out.round(4)) 

[2.48298614 1.06484103 1.49274094] [2.48298614 1.06484103 1.49274094] [ True  True  True]
[2.84290397 2.00141479 1.03128723] [2.84290397 2.00141479 1.03128723] [ True  True  True]
[1.72917101 1.89129523 2.81905332] [1.72917101 1.89129523 2.81905332] [ True  True  True]
[1.13833074 2.71066973 2.90375832] [1.13833074 2.71066973 2.90375832] [ True  True  True]
[2.60491339 2.06040844 1.83190064] [2.60491339 2.06040844 1.83190064] [ True  True  True]
[2.43085511 2.44070251 2.99944293] [2.43085511 2.44070251 2.99944293] [ True  True  True]
[2.65715894 1.17668211 1.53203542] [2.65715894 1.17668211 1.53203542] [ True  True  True]
[1.35047719 1.17232089 3.03529087] [1.35047719 1.17232089 3.03529087] [ True  True  True]
[1.60379403 1.88811651 1.35566381] [1.60379403 1.88811651 1.35566381] [ True  True  True]
[2.12680618 3.01411359 2.46566624] [2.12680618 3.01411359 2.46566624] [ True  True  True]
[3.08487961 1.19153305 2.62116847] [3.08487961 1.19153305 2.62116847] [ True  True  True]
[1.0865435

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [216]:
inputs, target = For_model.generate_maps()
#mot_pos = np.array([alpha,theta,radius,dists])
#n= np.random.randint(len(inputs))
pred = model(torch.Tensor(inputs)).detach().numpy()

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [221]:
For_model.orientation = True

## 2D TEST

In [9]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(100,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs,target = target,inputs

In [10]:
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=15, depth=100)
#model = NeuralNetworkSinusoid(n_in, n_out)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
for n in range(100):
    size = 500
    t = np.random.randint(-10,370,size=size)
    r = np.random.randint(150,size=size)
    inputs = np.array([r,t]).T

    x = np.sin(np.radians(t))*r
    y = np.cos(np.radians(t))*r
    target = np.array([x,y]).T
    
    train_loop(model, inputs, target, distance_loss,optimizer)
        
    if n % 10 == 0:
        optimizer.param_groups[0]["lr"] = optimizer.param_groups[0]["lr"]*0.5        

50.13558605194092
41.69785896301269
27.24435827255249
20.57340579986572
14.285087652206421
11.445655641555787
10.45282082080841
10.098252668380738
7.19429224729538
6.440871031284332
6.51929044008255
2.7458794486522673
3.3659899294376374
3.504760512113571
3.526837866306305
3.1385426807403562
3.0256880462169646
2.843242418766022
2.5200929391384124
2.5201621651649475
2.930586656332016
1.8759654599428177
1.187305451631546
2.156197493672371


KeyboardInterrupt: 

In [300]:
n= np.random.randint(len(inputs))
xy = target[n]
pred = model(torch.Tensor(inputs[n:n+1])).detach().numpy()
print(inputs[n])
print(target[n])
print(pred.round(2))

[ 43 163]
[ 12.5719833  -41.12110451]
[[ 12.65 -41.17]]


In [16]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(150,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs, target = target, inputs

pred = model(torch.Tensor(inputs)).detach().numpy()



r = inputs[:,0]
t = inputs[:,1]

x = target[:,0]
y = target[:,1]

x1 = r*np.sin(np.radians(t))
y1 = r*np.cos(np.radians(t))

x2 = pred[:,0]
y2 = pred[:,1]

x2 = r*np.sin(np.radians(x2))
y2 = r*np.cos(np.radians(y2))

plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [13]:
size = 1000

pred = model(torch.Tensor(inputs)).detach().numpy()

x = target[:,0]
y = target[:,1]

x2 = pred[:,0]
y2 = pred[:,1]


plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [522]:
    def forward(self,x_in):
                
        x_next = self.layer_first(x_in)
    
        out_list = [x_in,x_next]
        for n, block in enumerate(self.blocks):
            x_next = block(x_next) 
            out_list.append(x_next)  
            
        x = torch.cat(out_list, dim=1)
        
        x = F.leaky_relu(x)
        print(x)

        out = self.layer_last(x)
        
        return out